In [1]:
#!pip install ipyleaflet
#!jupyter nbextension enable --py --sys-prefix ipyleaflet
#!pip install folium
#Run using: jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [2]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import numpy as np
import pandas as pd

In [3]:
Baltimore=[39.29,-76.609383]

In [4]:
plotDay = '2015-04-28'

In [6]:
calls = pd.read_csv('../Data/911_calls_for_service_clean_final.csv',index_col='callDateTime',parse_dates=True)

In [7]:
calls[plotDay].priority.value_counts()

Medium           2471
High              944
Low               872
Non-Emergency     152
Emergency           2
Name: priority, dtype: int64

In [8]:
w = {'Out of Service':0.0,'Non-Emergency':0.2,'Low':0.4,'Medium':0.6,'High':0.8,'Emergency':1.0}
calls['pWeight'] = calls.priority.apply(lambda x: w[x])

In [9]:
calls.head()

,priority,district,description,callNumber,incidentLocation,location,lat,lon,pWeight
callDateTime,,,,,,,,,
2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)",39.289930,-76.612346,0.6
2015-07-13 10:47:00,Medium,CD,911 NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)",39.290674,-76.607160,0.6
2015-07-13 10:42:00,Medium,CD,911 NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)",39.289891,-76.612072,0.6
2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)",39.298516,-76.618475,0.4
2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)",39.311213,-76.676315,0.6


In [10]:
plotData = calls.loc[plotDay]

In [11]:
heat = plotData.loc[:,['lat','lon','pWeight']]
heat = heat[heat.lat.isnull() == False].values

heat_cord = heat[:,0:2].tolist()
heat_desc = calls.loc[:,['priority','description']].values.tolist()
heat = heat.tolist()

In [12]:
m = folium.Map(Baltimore, tiles='openstreetmap', zoom_start=11)
m.save('City_of_Baltimore.html')

HeatMap(heat,name='calls Heat').add_to(m)

pColor = {'Out of Service':'Black','Non-Emergency':'Black','Low':'Blue','Medium':'orange','High':'red','Emergency':'darkred'}

for loc in range(len(heat_cord)):
    desc = folium.Popup(heat_desc[loc][0]+ ': ' + str(heat_desc[loc][1]), parse_html=True)
    folium.CircleMarker(radius=1, location=heat_cord[loc],popup=desc,color=pColor[heat_desc[loc][0]]).add_to(m)
m.add_child(folium.map.LayerControl(collapsed=False))
m.save(plotDay+'_Heatmap.html')

In [13]:
#Generate Time Lapse data of 24 hours for plotDay
heatTime = []
for t in range(24):
    c = plotData.loc[plotDay+' '+str(t)+':00:00':plotDay+' '+str(t)+':59:59',['lat','lon','pWeight']]
    c = c[c.lat.isnull() == False].values.tolist()
    heatTime.append(c)

In [14]:
m = folium.Map(Baltimore,tiles='stamentoner', zoom_start=12)
HeatMapWithTime(heatTime,name='Heat_time',auto_play=True,max_opacity=0.6,max_speed=1).add_to(m)
for loc in range(len(heat_cord)):
    desc = folium.Popup(heat_desc[loc][0]+ ': ' + str(heat_desc[loc][1]), parse_html=True)
    folium.CircleMarker(radius=0, location=heat_cord[loc],popup=desc,color=pColor[heat_desc[loc][0]]).add_to(m)
m.save(plotDay+'_Heatmap_timelapse.html')

In [16]:
m = folium.Map(Baltimore, tiles='stamentoner', zoom_start=11)
HeatMap(heat,name='calls').add_to(m)
m#.save('stamentoner_Heatmap.html')

In [17]:
m = folium.Map(Baltimore, tiles='Stamen Terrain', zoom_start=11)
HeatMap(heat,name='calls').add_to(m)
m#.save('stamenterrain_Heatmap.html')